<a href="https://colab.research.google.com/github/rag1799/CLIP/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 759.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00


First, we implement the models themselves. A simple Graph Convolutional Network (GCN) with 2 layers is already shown. Your first task will be to finish a Graph Attention Network (GAT) with variable number of layers, hidden_dim and heads as well as the option to define a dropout.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GATConv
import torch.nn.functional as F

# GNN classes need an __init__() and a forward() function
class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        # Our GCN has two convolutional layers with a hidden dimension of 16 (16 "neurons")
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # For each layer perform the convolution and activation in between
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        # Return
        return x

# Now implement a Graph Attention Network with a variable number of layers, attention heads and hidden dimension - multiply the hidden dimension by the number of attention heads and set the attention heads to 1 for the output layer
class GAT(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_dim = 8, num_layers = 2, dropout = 0.1, heads = 8):
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        # GATConv(in_channels, hidden_dim, heads=heads, dropout=dropout)  # Multi-head attention

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # To prevent overfitting, you can artificially turn some neurons off, this is called "dropout"
        # Perform a dropout (torch.dropout) after every activation
        pass

Next comes the actual training of our networks. Typically, a data set is split into training set, validation set and test set. The training set is used for training and during training is validated using the validation set to reduce overfitting. Final performance is tested using the test set. Most things are already implemented here, you can add an early stopping mechanism if the model does not improve after a fixed amount of steps.



In [ ]:
from sklearn.metrics import accuracy_score

def train(model, optimizer, data, criterion):
    model.train()
    optimizer.zero_grad() # make previous gradients 0
    # Forward pass: model(data)
    out = model(data)

    # Calculate loss and perform backpropagation
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()

    # Optimization step
    optimizer.step()

    with torch.no_grad():
        pred = out[data.train_mask].argmax(dim=1) # filter by training data, get the class with the highest predicted value
        acc = accuracy_score(data.y[data.train_mask].cpu(), pred.cpu()) # get the accuracy versus the true classes

    return loss.item(), acc # loss is a tensor, convert to number

def validate(model, data, criterion): # validate on validation set - this works similar to the training, just with .val_mask (and you dont need gradient tracking, so you can use torch.no_grad for everything)
    model.eval()
    with torch.no_grad():
        # out =
        # loss =
        # pred =
        # acc =

    return loss.item(), acc

def evaluate(model, data): # evaluate on test set - this time with .test_mask
    model.eval()
    with torch.no_grad():
        # out =
        # pred =
        # acc =

    return acc

def train_and_validate(model, optimizer, data, criterion, epochs, patience, delta):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []
    best_val_acc = 0
    patience_counter = 0

    for epoch in range(epochs):
        train_loss, train_acc = train(model, optimizer, data, criterion)
        val_loss, val_acc = validate(model, data, criterion)

        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        # Implement the patience: If there is not at least a *delta* difference in *val_acc* in *patience* steps, stop training
        # if val_acc > best_val_acc + delta:

    test_acc = evaluate(model, data) # test set accuracy

    return train_losses, train_accs, val_losses, val_accs, test_acc

IndentationError: expected an indented block after 'with' statement on line 24 (ipython-input-3845810250.py, line 30)

This is a plotting function to show the losses and accuracy over time

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def create_training_plots(model_names, train_losses, train_accs, val_losses, val_accs, save_path):
    fig, axes = plt.subplots(2, len(model_names), figsize=(18, 12))
    colors = sns.color_palette('colorblind', len(model_names)) # works for at most 10 models
    for i, (model_name, color) in enumerate(zip(model_names, colors)):
      result = model_name[i]
      if len(model_names) > 1:
        axes[0, i].plot(train_losses[i], color=color, label='Train Loss')
        axes[0, i].plot(val_losses[i], color=color, linestyle='--', label='Val Loss')
        axes[0, i].set_title(f'{model_name} - Loss Curves')
        axes[0, i].set_xlabel('Epoch')
        axes[0, i].set_ylabel('Loss')
        axes[0, i].legend()
        axes[0, i].grid(True)
        axes[1, i].plot(train_accs[i], color=color, label='Train Acc')
        axes[1, i].plot(val_accs[i], color=color, linestyle='--', label='Val Acc')
        axes[1, i].set_title(f'{model_name} - Accuracy Curves')
        axes[1, i].set_xlabel('Epoch')
        axes[1, i].set_ylabel('Accuracy')
        axes[1, i].legend()
        axes[1, i].grid(True)
      else:
        axes[0].plot(train_losses[i], color=color, label='Train Loss')
        axes[0].plot(val_losses[i], color=color, linestyle='--', label='Val Loss')
        axes[0].set_title(f'{model_name} - Loss Curves')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].legend()
        axes[0].grid(True)
        axes[1].plot(train_accs[i], color=color, label='Train Acc')
        axes[1].plot(val_accs[i], color=color, linestyle='--', label='Val Acc')
        axes[1].set_title(f'{model_name} - Accuracy Curves')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Accuracy')
        axes[1].legend()
        axes[1].grid(True)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

Next, we get into the data. The Cora citation network is stored in _Planetoid_ and we can get some information about the graph

In [ ]:
# Load dataset (Cora citation network)
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]  # Get the graph data (Cora)

print(dataset.num_classes) # shows the number of classes in the dataset (research areas)

attributes = [attr for attr in dir(data)
              if not callable(getattr(data, attr))
              and not attr.startswith('_')]
attributes.extend([attr for attr in data._store
              if not callable(getattr(data, attr))
              and not attr.startswith('__')])

print(attributes)

# These are attributes of our data set. Use this to get the size of the graph in terms of nodes and edges, as well as the size of the training, validation and test sets


Finally, we set up the model, train it on the Cora network data and plot the training process as well as get a final label prediction accuracy

In [ ]:
# Create a model instance - what are the in_channels and what are the out_channels? Try a couple of different parameters for your GAT to increase the accuracy
gcn_model = GCN(dataset.num_node_features, dataset.num_classes)
# gat_model =

optimizer_gcn = optim.Adam(gcn_model.parameters(), lr=0.001) # Adam optimizer for minimizing the loss function, GAT can use the same
# optimizer_gat =

criterion = nn.CrossEntropyLoss() # Loss function to minimize - when using CrossEntropyLoss, the forward function should return raw logits, when using other loss, the forward should return F.log_softmax(x) for multi-class classification

gcn_train_loss, gcn_train_accs, gcn_val_loss, gcn_val_accs, gcn_accuracy = train_and_validate(gcn_model, optimizer_gcn, data, criterion, epochs=200, patience=10, delta=0.001)

# update the plot to show both GCN and GAT
create_training_plots(["GCN"], [gcn_train_loss], [gcn_train_accs], [gcn_val_loss], [gcn_val_accs], "plot_GCN.png")
print(gcn_accuracy)